In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/sample_submission.csv
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/test.csv
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/LICENSE
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/README.md
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/meta/esc50.csv
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/meta/esc50-human.xlsx
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/audio/5-257349-A-15.wav
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/audio/5-195557-A-19.wav
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/audio/2-122820-B-36.wav
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/audio/1-115920-A-22.wav
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/audio/1-172649-C-40.wav
/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master/audio/

In [2]:
import os
import random
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader


GENRES = [
    "blues", "classical", "country", "disco", "hiphop",
    "jazz", "metal", "pop", "reggae", "rock"
]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:
class MessyMashupTrainDataset(Dataset):
    def __init__(self, root, noise_root, sr=22050, duration=5):
        self.root = root
        self.noise_root = noise_root
        self.sr = sr
        self.target_len = sr * duration

        self.noise_files = [
            os.path.join(noise_root, "audio", f)
            for f in os.listdir(os.path.join(noise_root, "audio"))
        ]

        self.samples = []
        for genre in GENRES:
            genre_path = os.path.join(root, genre)
            for song in os.listdir(genre_path):
                self.samples.append(genre)

    def __len__(self):
        return len(self.samples)

    def _load_audio(self, path):
        audio, _ = librosa.load(path, sr=self.sr, mono=True)
        if len(audio) > self.target_len:
            audio = audio[:self.target_len]
        else:
            audio = np.pad(audio, (0, self.target_len - len(audio)))
        return audio

    def __getitem__(self, idx):
        genre = self.samples[idx]
        genre_path = os.path.join(self.root, genre)

        stems = ["drums.wav", "bass.wav", "vocals.wav", "others.wav"]
        mix = np.zeros(self.target_len, dtype=np.float32)

        loaded = False

        for stem in stems:
            song = random.choice(os.listdir(genre_path))
            stem_path = os.path.join(genre_path, song, stem)

            if not os.path.exists(stem_path):
                continue  # 🔴 skip missing stem

            audio = self._load_audio(stem_path)
            loaded = True

            if random.random() < 0.3:
                rate = random.uniform(0.8, 1.2)
                audio = librosa.effects.time_stretch(audio, rate=rate)
                if len(audio) > self.target_len:
                    audio = audio[:self.target_len]
                else:
                    audio = np.pad(audio, (0, self.target_len - len(audio)))

            mix += random.uniform(0.5, 1.2) * audio

        # 🔁 If no stems loaded, retry with a different index
        if not loaded:
            return self.__getitem__((idx + 1) % len(self))

        # Noise injection
        if random.random() < 0.5:
            noise = self._load_audio(random.choice(self.noise_files))
            mix += 0.3 * noise

        mix /= (np.max(np.abs(mix)) + 1e-6)

        mel = librosa.feature.melspectrogram(
            y=mix, sr=self.sr, n_mels=128, n_fft=2048, hop_length=512
        )
        mel = librosa.power_to_db(mel, ref=np.max)

        mel = torch.tensor(mel).unsqueeze(0).float()
        label = torch.tensor(GENRES.index(genre)).long()

        return mel, label


In [4]:
class AudioCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(128, 10)

    def forward(self, x):
        x = self.net(x)
        x = x.squeeze(-1).squeeze(-1)
        return self.fc(x)


In [5]:
train_dataset = MessyMashupTrainDataset(
    root="/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/genres_stems",
    noise_root="/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/ESC-50-master"
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

model = AudioCNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "audio_cnn.pth")
print("✅ Model saved as audio_cnn.pth")


Epoch 1/10 | Loss: 2.1370
Epoch 2/10 | Loss: 1.9942
Epoch 3/10 | Loss: 1.9055
Epoch 4/10 | Loss: 1.8803
Epoch 5/10 | Loss: 1.8200
Epoch 6/10 | Loss: 1.7521
Epoch 7/10 | Loss: 1.7873
Epoch 8/10 | Loss: 1.6930
Epoch 9/10 | Loss: 1.6935
Epoch 10/10 | Loss: 1.6786
✅ Model saved as audio_cnn.pth


In [6]:
class MashupTestDataset(Dataset):
    def __init__(self, csv_path, base_dir, sr=22050, duration=5):
        self.df = pd.read_csv(csv_path)
        self.base_dir = base_dir
        self.sr = sr
        self.target_len = sr * duration

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # ✅ prepend Kaggle dataset root
        audio_path = os.path.join(self.base_dir, row["filename"])

        audio, _ = librosa.load(audio_path, sr=self.sr, mono=True)

        if len(audio) > self.target_len:
            audio = audio[:self.target_len]
        else:
            audio = np.pad(audio, (0, self.target_len - len(audio)))

        mel = librosa.feature.melspectrogram(
            y=audio,
            sr=self.sr,
            n_mels=128,
            n_fft=2048,
            hop_length=512
        )
        mel = librosa.power_to_db(mel, ref=np.max)

        mel = torch.tensor(mel).unsqueeze(0).float()
        return mel, row["id"]


In [7]:
BASE_DIR = "/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup"

test_dataset = MashupTestDataset(
    csv_path=f"{BASE_DIR}/test.csv",
    base_dir=BASE_DIR
)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [8]:
model = AudioCNN().to(DEVICE)
model.load_state_dict(torch.load("audio_cnn.pth", map_location=DEVICE))
model.eval()

test_dataset = MashupTestDataset("/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/test.csv" , "/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup" )
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

predictions = []

with torch.no_grad():
    for x, ids in test_loader:
        x = x.to(DEVICE)
        preds = torch.argmax(model(x), dim=1).cpu().numpy()

        for i, p in zip(ids, preds):
            predictions.append((i, GENRES[p]))


In [9]:
submission = pd.DataFrame(predictions, columns=["id", "genre"])
submission.to_csv("submission.csv", index=False)

print("✅ submission.csv generated")


✅ submission.csv generated
